In [4]:
!pipreqs --force .


INFO: Not scanning for jupyter notebooks.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in .\requirements.txt


In [11]:
!pip install torchvision

^C


In [12]:
# Cell 1: Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
from torch.cuda import amp  # For mixed precision training

In [13]:
# Cell 2: Dataset Class
class MovementDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = ['basic_movements', 'grasping_movements']
        self.file_list = []
        self.labels = []
        
        for class_idx, class_name in enumerate(self.classes):
            class_path = os.path.join(data_dir, class_name)
            files = os.listdir(class_path)
            self.file_list.extend([os.path.join(class_path, f) for f in files])
            self.labels.extend([class_idx] * len(files))
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [14]:
# Cell 3: Optimized ResNet Model
class ResNetModel(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetModel, self).__init__()
        
        # Load pretrained ResNet18 instead of ResNet50
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        
        # Freeze only early layers
        for name, param in self.resnet.named_parameters():
            if "layer4" not in name and "fc" not in name:
                param.requires_grad = False
            
        # Enhanced classifier head
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        return self.resnet(x)

In [15]:
# Cell 4: Enhanced Data Transforms
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [16]:
# Cell 5: Mixup Function
def mixup_data(x, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [17]:
# Cell 6: Optimized Training Function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_model(model, train_loader, num_epochs=100):
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(
        model.parameters(),
        lr=0.001,  # Increased learning rate
        weight_decay=0.01,
        betas=(0.9, 0.999)
    )
    
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.001,
        epochs=num_epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.1,
        div_factor=25.0,
        final_div_factor=1000.0
    )
    
    scaler = amp.GradScaler()
    
    best_acc = 0
    patience = 5
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(train_loader)
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            with amp.autocast():
                if np.random.random() > 0.5:
                    inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
                    outputs = model(inputs)
                    loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
                else:
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
            
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            pbar.set_description(
                f'Epoch {epoch+1}/{num_epochs} | '
                f'Loss: {loss.item():.4f} | '
                f'Acc: {100.*correct/total:.2f}% | '
                f'LR: {scheduler.get_last_lr()[0]:.6f}'
            )
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100. * correct / total
        
        print(f"\nEpoch Summary:")
        print(f"Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'acc': epoch_acc,
            }, 'best_model.pth')
            print(f"New best accuracy! Model saved.")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(".")
                
    
    return best_acc

In [18]:
# Cell 7: Main Execution
if __name__ == "__main__":
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    if torch.cuda.is_available():
        print(f"Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Create model
    model = ResNetModel(num_classes=2).to(device)
    
    # Create dataset and loader
    batch_size = 32  # Reduced batch size
    train_dataset = MovementDataset(data_dir='movements', transform=transform_train)
    
    # Print class distribution
    print("\nClass distribution:")
    class_counts = [0, 0]
    for _, label in train_dataset:
        class_counts[label] += 1
    print(f"Basic movements: {class_counts[0]}")
    print(f"Grasping movements: {class_counts[1]}")
    print(f"Ratio: {class_counts[0]/class_counts[1]:.2f}")
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2 if torch.cuda.is_available() else 0,
        pin_memory=torch.cuda.is_available()
    )
    
    # Train model
    best_accuracy = train_model(model, train_loader)
    print(f"\nTraining completed! Best accuracy: {best_accuracy:.2f}%")

Using device: cpu

Class distribution:


C:\Users\kkrrt\AppData\Local\Temp\ipykernel_19876\898063188.py:23: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
c:\Users\kkrrt\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Basic movements: 3403
Grasping movements: 3403
Ratio: 1.00


  0%|          | 0/213 [00:00<?, ?it/s]C:\Users\kkrrt\AppData\Local\Temp\ipykernel_19876\898063188.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
c:\Users\kkrrt\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 1/100 | Loss: 0.7524 | Acc: 55.61% | LR: 0.000064: 100%|██████████| 213/213 [12:33<00:00,  3.54s/it]



Epoch Summary:
Loss: 0.7002 | Accuracy: 55.61%
New best accuracy! Model saved.


Epoch 2/100 | Loss: 0.6391 | Acc: 55.90% | LR: 0.000089:  46%|████▌     | 97/213 [05:22<06:25,  3.32s/it]


KeyboardInterrupt: 

In [8]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os

# 1) Define the exact same ResNet18-based model used during training.
def create_model(num_classes=2):
    # Create a baseline resnet18 (no pretrained weights if you trained from scratch).
    # If you originally used pretrained=True, then pass it here. 
    model = models.resnet18(weights=None)
    
    # Adjust the final layer to match your num_classes
    # By default, resnet18's final layer is nn.Linear(512, 1000)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    
    return model

# 2) Load the checkpoint
def load_model(checkpoint_path="best_model.pth", device="cpu", num_classes=2):
    # Instantiate the model architecture
    model = create_model(num_classes=num_classes).to(device)
    
    # Load the entire checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    # Note: if you only saved model.state_dict(), load it directly:
    # model.load_state_dict(checkpoint)
    # Otherwise, if you saved a dict with 'model_state_dict', do:
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Switch to eval mode
    model.eval()
    
    return model



In [9]:
# Cell 4: Optional – Validate on a Folder

def validate_model(model, data_dir, device="cpu", batch_size=4):
    """
    Validates the model on a folder structured as:
      data_dir/class0/*.jpg
      data_dir/class1/*.jpg
      ...
    Returns accuracy in percentage, plus class names.
    """
    # Match the transforms used during training
    val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    val_dataset = datasets.ImageFolder(root=data_dir, transform=val_transforms)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    correct = 0
    total = 0
    
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    accuracy = 100.0 * correct / total
    return accuracy, val_dataset.classes

print("Validate Model function defined.")

Validate Model function defined.


In [10]:
# Cell 5: Optional – Single-Image Prediction

def predict_single_image(model, image_path, device="cpu", class_names=None):
    """
    Loads a single image, applies transforms, and prints the predicted class.
    """
    # Match the transforms from training
    test_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    # Load & transform
    img = Image.open(image_path).convert('RGB')
    img_tensor = test_transforms(img).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted_idx = torch.max(outputs, 1)
    
    if class_names:
        print(f"Predicted Class: {class_names[predicted_idx.item()]}")
    else:
        print(f"Predicted Class Index: {predicted_idx.item()}")

print("Single Image Prediction function defined.")

Single Image Prediction function defined.


In [11]:
# Cell 6: Main Usage Example

if __name__ == "__main__":
    # 1) Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # 2) Load model from checkpoint
    best_model_path = "best_model.pth"
    if os.path.exists(best_model_path):
        model = load_model(checkpoint_path=best_model_path, device=device, num_classes=2)
        print("Loaded best model successfully!")
    else:
        print(f"Checkpoint not found at {best_model_path}")
        raise SystemExit
    
    # 3) (Optional) Validate on a dataset folder
    validation_dir = "movements/basic_movements"  # e.g. "movements/val"
    if os.path.exists(validation_dir):
        accuracy, class_list = validate_model(model, validation_dir, device=device, batch_size=4)
        print(f"Validation Accuracy on '{validation_dir}': {accuracy:.2f}%")
        print(f"Class names: {class_list}")
    else:
        print(f"Validation directory '{validation_dir}' not found. Skipping validation.")
    
    # 4) (Optional) Predict a single image
    test_image_path = "angle_1_Basic_movements_spectrogram_3.png"
    if os.path.exists(test_image_path):
        # Provide class names if you have them
        your_class_names = ["basic movement", "grasping movement"]  # Update as needed
        predict_single_image(model, test_image_path, device=device, class_names=your_class_names)
    else:
        print(f"No test image at {test_image_path}. Skipping single-image prediction.")

Using device: cpu


C:\Users\kkrrt\AppData\Local\Temp\ipykernel_19876\1115107449.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=devic

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer2.0.conv1.weight", "layer2.0.bn1.weight", "layer2.0.bn1.bias", "layer2.0.bn1.running_mean", "layer2.0.bn1.running_var", "layer2.0.conv2.weight", "layer2.0.bn2.weight", "layer2.0.bn2.bias", "layer2.0.bn2.running_mean", "layer2.0.bn2.running_var", "layer2.0.downsample.0.weight", "layer2.0.downsample.1.weight", "layer2.0.d